- Creating a new Dash App using the yfinance API as the data pipeline

- Importing the necessary libraries to execute the application

In [1]:
import numpy as np
import dash
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import plotly.express as px
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import openpyxl
import warnings
warnings.filterwarnings("ignore")


symbols_df = pd.read_excel(io=r"C:\Users\Gabriel Flynn\OneDrive\OneDrive - University of Texas at El Paso\Documents\Python Projects\Yfinance_Stock_Data_Analysis\yfin_dash_app_api_backend\Sp500Symbols.xlsx",
                        sheet_name='Table 1',engine='openpyxl')
symbols = symbols_df['Symbol'].to_list()

period = ["1d", "5d", "1mo", "3mo", "6mo", "1y", "2y", "5y", "10y", "ytd", "max"]
intervals = ["1m", "2m", "5m", "15m", "30m", "60m", "90m", "1h", "1d", "5d", "1wk", "1mo", "3mo"]

- Utilizing polygon io to provide a comprehensive list of stock symbols

- Creating the list for date filters to be used in the line charts

- Creating he dash board frame work

In [2]:
# %% [markdown]
# - Creating the dashboard framework

# %%
app = dash.Dash(__name__)
server = app.server

app.layout = html.Div([
    # Title
    html.H1('Welcome to the Stock Analytics Dashboard',
            style={'textAlign': 'center', 'padding': '20px', 'color': '#1a1a1a'}),

    # Dropdown Filters Row
    html.Div([
        html.Div([
            html.Label('Select Stock'),
            dcc.Dropdown(symbols, 'AAPL', id='stock_symbols', clearable=False)
        ], style={'flex': 1, 'marginRight': '10px'}),

        html.Div([
            html.Label('Select Period'),
            dcc.Dropdown(period, '5y', id='period', clearable=False)
        ], style={'flex': 1, 'marginRight': '10px'}),

        html.Div([
            html.Label('Select Interval'),
            dcc.Dropdown(intervals, '1d', id='intervals', clearable=False)
        ], style={'flex': 1})
    ], style={
        'display': 'flex', 'flexDirection': 'row',
        'justifyContent': 'space-between',
        'padding': '0px 20px 20px 20px'
    }),

    html.Hr(),

    # Main Graph Container for a 50/50 split
    html.Div([
        # Price Trend Line - Left half of the screen
        html.Div([
            html.H3("📈 Stock Price Trend", style={'textAlign': 'left'}),
            dcc.Graph(id='trend_line')
        ], style={'flex': '1 1 0%'}),

        # RSI and MACD graphs - Right half of the screen, stacked
        html.Div([
            # RSI Graph
            html.Div([
                html.H3("📈 Relative Strength Index (RSI)", style={'textAlign': 'left'}),
                dcc.Graph(id='RSI')
            ], style={'flex': '1 1 0%',"height":"400px"}), # Use flex on the child to ensure it takes available space

            # MACD Graph
            html.Div([
                html.H3("📈 MACD Signal Line", style={'textAlign': 'left'}),
                dcc.Graph(id='MACD')
            ], style={'flex': '1 1 0%',"height":"400px"}) # Use flex on the child to ensure it takes available space
        ],
        style={
            'display': 'flex',
            'flexDirection': 'row', # Stacks RSI and MACD vertically
            'flex': '1 1 0%', # Allows this container to take up half the width
            "height":"400px", 
            'gap': '20px',            # Adds a consistent gap between the RSI and MACD graphs
        }),

    ], style={
        'display': 'flex',
        'flexDirection': 'column',  # Arranges the two main Divs horizontally
        'padding': '20px',
        'gap': '20px'            # Adds spacing between the two columns
    }),


    ##News Inquiry Table
    html.Div([
        html.Div([
            html.H3("📰 Latest News", style={'textAlign': 'left'}),
            html.Div(id='news_table')
        ], style={'flex': 1})
    ], style={
        'display': 'flex',
        'flexDirection': 'row',
        'padding': '20px'
    }),

    html.Hr(),

    # Distribution and Forecast Table
    html.Div([
        
        html.Div([
            html.H3("📈 Price & Volume Distribution"),
            html.Div(id='price_dis_table'),
            html.H3("🔮 Earnings Forecast"),
            html.Div(id='forecast')
        ], style={'flex': 1, 'flexDirection':'column','marginRight': '20px'}),
        
        html.Div([html.H3("📊 Historical Financials"),
                  html.Div(id='historical_financials'),
                  html.H3(children='📊 Quarterly Earnings Per Share', style={',arginTop':'20px'}),
                  dcc.Graph(id='EPS', style={'marginTop': '20px','height':'100%'})
                  ], 
                 style={'flex': 1,'flexDirection':'column','marginRight':'20px','height':'400px'})
    ], style={
        'display': 'flex',
        'flexDirection': 'row',
        'padding': '20px'
    })
], style={
    'backgroundColor': '#f5f5f5',
    'fontFamily': 'Arial, sans-serif',
    'paddingBottom': '40px'
})

- Creating the graph call backs

In [3]:
@callback(
    Output("news_table", "children"),
    Output("historical_financials", "children"),
    Input("stock_symbols", "value"),
)
def create_table(ticker: str):
    stock = yf.Ticker(ticker)

    # News (robust parsing)
    news = stock.get_news(count=10, tab="news", proxy=None) or []
    # Pull nested content dicts if present
    records = []
    for item in news:
        content = item.get("content", {})
        # Some fields may be at top-level; prefer content then fallback
        title = content.get("title") or item.get("title")
        summary = content.get("summary") or item.get("summary")
        pubDate = content.get("pubDate") or item.get("pubDate")
        url = None
        cu = content.get("canonicalUrl") or item.get("canonicalUrl")
        if isinstance(cu, dict):
            url = cu.get("url")
        elif isinstance(cu, str):
            url = cu
        records.append({"TITLE": title, "SUMMARY": summary, "PUBDATE": pubDate, "URL": url})

    news_df = pd.DataFrame(records)
    articles = dash_table.DataTable(
        columns=[{"name": c, "id": c} for c in news_df.columns],
        data=news_df.to_dict("records"),
        cell_selectable=True,
        page_size=5,
        style_table={
            "overflowX": "auto",
            "overflowY": "auto",
            "maxHeight": "400px",
            "border": "1px solid #ccc",
        },
        style_cell={
            "textAlign": "left",
            "whiteSpace": "normal",
            "minWidth": "150px",
            "width": "200px",
            "maxWidth": "300px",
            "padding": "6px",
            "overflowX": "auto",
            "overflowY": "auto",
        },
        style_header={
            "backgroundColor": "black",
            "color": "white",
            "fontWeight": "bold",
            "textAlign": "left",
            "overflowX": "auto",
            "overflowY": "auto",
        },
    )

    # Historical financials (guard for empty)
    financials = stock.financials

    financials = financials.T

    financials.index = pd.to_datetime(financials.index).year

    key_financials = financials[['Total Revenue', 'Gross Profit', 'Operating Income', 'Net Income','Normalized Income']]

    key_financials.reset_index(inplace=True)

    key_financials.rename(columns={'index':'Year'}, inplace=True)
    
    financials_table = dash_table.DataTable(
        columns=[{'name':col, 'id':col} for col in key_financials.columns],
        data=key_financials.to_dict('records'),
        cell_selectable=True,
        style_table={
        'overflowX': 'auto',         # Scroll horizontally if needed
        'overflowY': 'auto',         # Scroll vertically inside the box
        'maxHeight': '400px',        # Set max visible height
        'border': '2px solid #ccc'   # Optional: makes the box look nice
    },
    
    style_cell={
        'textAlign': 'left',
        'whiteSpace': 'normal',
        'minWidth': '150px',
        'width': '200px',
        'maxWidth': '300px',
        'padding': '6px',
        'overflowX':'auto',
        'overflowY':'auto',
    },

    style_header={
        'backgroundColor': 'black',
        'color': 'white',
        'fontWeight': 'bold',
        'textAlign': 'left',
        'overflowX':'auto',
        'overflowY':'auto'
    }
)

    return articles, financials_table

# -------- Trend line, RSI, forecast, EPS, MACD --------
@callback(
    Output("trend_line", "figure"),
    Output("RSI", "figure"),
    Output("forecast", "children"),
    Output("EPS", "figure"),
    Output("MACD", "figure"),
    Input("stock_symbols", "value"),
    Input("period", "value"),
    Input("intervals", "value"),
)
def trend_chart(ticker: str, period: str, intervals: str):
    company = yf.Ticker(ticker)
    df = yf.download(tickers=ticker, period=period, interval=intervals)

    if df.empty:
        return go.Figure(), go.Figure(), html.Div("No data."), go.Figure(), go.Figure()

    # Flatten multi-index columns if needed
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [c[0] for c in df.columns]

    # Moving averages (guard short series)
    df["50_DAY_MA"] = df["Close"].rolling(window=50, min_periods=1).mean()
    df["200_DAY_MA"] = df["Close"].rolling(window=200, min_periods=1).mean()

    # RSI (EMA version)
    rs = df[["Close"]].copy()
    rs["Δ"] = rs["Close"].diff()
    gains = rs["Δ"].clip(lower=0)
    losses = (-rs["Δ"].clip(upper=0))
    rs["avg_gain"] = gains.ewm(com=14 - 1, adjust=False).mean()
    rs["avg_loss"] = losses.ewm(com=14 - 1, adjust=False).mean()
    rs["RSI"] = 100 - (100 / (1 + (rs["avg_gain"] / rs["avg_loss"].replace(0, np.nan))))
    rs["RSI"] = rs["RSI"].round(2)

    # EPS (guard empty)
    earns = getattr(company, "earnings_history", None)
    eps_fig = go.Figure()
    if isinstance(earns, pd.DataFrame) and not earns.empty:
        try:
            earns = earns.copy()
            earns.index = pd.to_datetime(earns.index).date
            eps_fig.add_trace(go.Bar(x=earns.index, y=earns.get("epsActual"), name="EPS Actual"))
            eps_fig.add_trace(go.Bar(x=earns.index, y=earns.get("epsEstimate"), name="EPS Estimate"))
            eps_fig.update_layout(
                title={"text":"<b>Actual vs Estimate Earnings per Share<b>",
                      "x":.5,
                      "y":.95},
                xaxis_title="<b>Date<b>",
                yaxis_title="<b>EPS<b>",
                barmode="group",
                legend=dict(title="Legend", orientation="h", yanchor="bottom", y=1.0, xanchor="right", x=1),
            )
        except Exception:
            pass

    # RSI figure
    rsi_fig = go.Figure()
    rsi_fig.add_trace(go.Scatter(x=rs.index, y=rs["RSI"], name="RSI"))
    rsi_fig.update_layout(
        title={"text":"<b>Relative Strength Index (RSI)<b>",
              "x":.5,
              "y":.95},
        xaxis_title="<b>DATE<b>",
        yaxis_title="<b>RSI<b>",
        yaxis=dict(range=[0, 100]),
        shapes=[
            dict(type="line", yref="y", y0=70, y1=70, xref="x", x0=df.index[0], x1=df.index[-1]),
            dict(type="line", yref="y", y0=30, y1=30, xref="x", x0=df.index[0], x1=df.index[-1]),
        ],
    )

    # MACD
    macd_df = df[["Close"]].copy()
    macd_df["EMA_12"] = macd_df["Close"].ewm(span=12, adjust=False).mean()
    macd_df["EMA_26"] = macd_df["Close"].ewm(span=26, adjust=False).mean()
    macd_df["MACD_LINE"] = macd_df["EMA_12"] - macd_df["EMA_26"]
    macd_df["SIGNAL_LINE"] = macd_df["MACD_LINE"].ewm(span=9, adjust=False).mean()
    macd_df["HISTOGRAM"] = macd_df["MACD_LINE"] - macd_df["SIGNAL_LINE"]

    macd_fig = go.Figure()
    macd_fig.add_trace(go.Scatter(x=macd_df.index, y=macd_df["MACD_LINE"], name="MACD"))
    macd_fig.add_trace(go.Scatter(x=macd_df.index, y=macd_df["SIGNAL_LINE"], name="Signal"))
    macd_fig.add_trace(go.Bar(x=macd_df.index, y=macd_df["HISTOGRAM"], name="Histogram"))
    macd_fig.update_layout(title={"text":"<b>MACD<b>",
                                 "x":.5,
                                 "y":.95},
                          yaxis_title="<b>MACD<b>",
                          xaxis_title="<b>DATE<b>")

    # Trend line
    trend_fig = go.Figure()
    trend_fig.add_trace(go.Scatter(x=df.index, y=df["Close"], name=f"{ticker} Close"))
    trend_fig.add_trace(go.Scatter(x=df.index, y=df["50_DAY_MA"], name="50-Day MA"))
    trend_fig.add_trace(go.Scatter(x=df.index, y=df["200_DAY_MA"], name="200-Day MA"))
    trend_fig.update_layout(title={"text":"<b>Closing Price Trend<b>"}, xaxis_title="<b>DATE<b>", yaxis_title="<b>PRICE<b>")

    # Forecast Table 
    stock = yf.Ticker(ticker)
    forecast_ticker = pd.DataFrame(stock.calendar).T
    forecast_ticker.rename(columns={0:'Projected Forecast'}, inplace=True)
    forecast_ticker.reset_index(inplace=True)
    forecast_ticker.rename(columns={'index':'Forecast Type'}, inplace=True)
    forecast_table = dash_table.DataTable(
        columns=[{'name': col, 'id': col} for col in forecast_ticker.columns],
        data=forecast_ticker.to_dict('records'),
        style_table={'overflowY': 'auto','minHeight':'100%'},
        style_cell={
            'textAlign': 'left',
            'whiteSpace': 'normal',
            'height': 'auto',
            'width': 'auto',
            'padding': '2px'
        },
        style_header={
            'backgroundColor': 'black',
            'color': 'white',
            'fontWeight': 'bold',
            'textAlign': 'center'
        }
    )

    return trend_fig, rsi_fig, forecast_table, eps_fig, macd_fig

# -------- Distribution table --------
@callback(
    Output("price_dis_table", "children"),
    Input("stock_symbols", "value"),
)
def dist_table(ticker):

    df = yf.download(tickers=ticker, period='5y')
    
    cols_new = []

    for x in df.columns:
        new = x[0]
        cols_new.append(new)
    
    df.columns = cols_new
    
    df_close_dist = df['Close'].describe()

    df_close_dist = df_close_dist.to_frame(name='Closing Price').round(2)
    
    df_close_dist.reset_index(inplace=True)
    
    df_close_dist.rename(columns={'index':'Statistics'}, inplace=True)
    
    df_volume_dist = df['Volume'].describe()
    
    df_volume_dist = df_volume_dist.to_frame(name='Volume').round(2)
    
    df_volume_dist.reset_index(inplace=True)
    
    df_volume_dist.rename(columns={'index':'Statistics'}, inplace=True)
    
    df_close_dist = pd.concat(objs=[df_close_dist,df_volume_dist[['Volume']]], axis=1, join='inner',ignore_index=False)
    
    previous_data = df.iloc[-1,[0,4]].to_frame().transpose()

    previous_data.rename(index={previous_data.index[0]:'previous data'}, inplace=True)
    
    previous_data.reset_index(inplace=True)
    
    previous_data.rename(columns={'index':'Statistics',
                                  'Close':'Closing Price'},
                                  inplace=True)
    
    previous_data['Closing Price'] = previous_data['Closing Price'].round(2)
    
    df_close_dist = pd.concat(objs=[df_close_dist, previous_data], axis=0,ignore_index=False)
    
    df_close_dist['Volume'] = df_close_dist['Volume'].apply(lambda x: f"{x:.2e}")
    
    
    return dash_table.DataTable(
        columns=[{'name':col, 'id':col} for col in df_close_dist.iloc[1:,:].columns],
        data=df_close_dist.iloc[1:,:].to_dict('records'),
        style_table={'overflowY': 'auto'},
        style_cell={'textAlign': 'left',
                    'whitespace':'normal',
                    'height':'auto',
                    'width':'100px',
                    'padding':'8px'},
        style_header={'backgroundColor': 'black',
                      'color': 'white',
                      'fontWeight': 'bold',
                      'textAlign': 'center'}
    )
if __name__ == "__main__":
    app.run(jupyter_mode='external',debug=False)

Dash app running on http://127.0.0.1:8050/


[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed

